<a href="https://colab.research.google.com/github/vijaykrishna0105/CSE508_Winter2024_A1_2019183/blob/main/CSE508_Winter2024_A2_2019183.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
save_directory = '/content/drive/MyDrive/IR_A2_files'


In [62]:
os.makedirs(save_directory, exist_ok=True)


In [2]:
!pip install tensorflow

import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
import requests
from io import BytesIO
from sklearn.preprocessing import StandardScaler


In [3]:
# Adjust the path as necessary
dataset_path = '/content/drive/MyDrive/A2_Data.csv'
df = pd.read_csv(dataset_path)


In [4]:
# Display the first 5 rows of the dataframe to understand its structure
print(df.head())

   Unnamed: 0                                              Image  \
0        3452  ['https://images-na.ssl-images-amazon.com/imag...   
1        1205  ['https://images-na.ssl-images-amazon.com/imag...   
2        1708  ['https://images-na.ssl-images-amazon.com/imag...   
3        2078  ['https://images-na.ssl-images-amazon.com/imag...   
4         801  ['https://images-na.ssl-images-amazon.com/imag...   

                                         Review Text  
0  Loving these vintage springs on my vintage str...  
1  Works great as a guitar bench mat. Not rugged ...  
2  We use these for everything from our acoustic ...  
3  Great price and good quality.  It didn't quite...  
4  I bought this bass to split time as my primary...  


In [5]:
def download_and_process_img(url):
    response = requests.get(url)
    img = load_img(BytesIO(response.content), target_size=(224, 224))
    img_array = img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)


In [6]:
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)


102967424/102967424 [==============================] - 1s 0us/step


In [7]:
# from PIL import UnidentifiedImageError

# for url in df['first_image_url']:
#     try:
#         if url:  # Check if URL is not None
#             processed_img = download_and_process_img(url)
#             features = model.predict(processed_img)
#             features_list.append(features.flatten())
#     except UnidentifiedImageError:
#         print(f"Failed to process image: {url}")
#         features_list.append(np.zeros((model.output.shape[1],)))  # Append zeros or handle otherwise


KeyError: 'first_image_url'

In [12]:
import ast  # For safely evaluating a string that contains a Python literal

# Function to extract the first URL from the 'Image' column
def extract_first_url(image_string):
    image_list = ast.literal_eval(image_string)  # Converts string to list
    if image_list:  # If the list is not empty
        return image_list[0]  # Return the first URL
    return None  # If the list is empty, return None

# Apply this function to each row in the 'Image' column
df['first_image_url'] = df['Image'].apply(extract_first_url)


In [31]:
features_list = []  # Initialize the list to store features

from PIL import UnidentifiedImageError

# Updated loop to process images from the 'first_image_url' column
for url in df['first_image_url']:
    try:
        if url:  # Check if URL is not None
            processed_img = download_and_process_img(url)
            features = model.predict(processed_img)
            features_list.append(features.flatten())
    except UnidentifiedImageError:
        print(f"Failed to process image: {url}")
        features_list.append(np.zeros((model.output.shape[1],)))  # Append zeros for missing or failed images


1/1 [==============================] - 0s 219ms/step
Failed to process image: https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg
1/1 [==============================] - 0s 214ms/step
Failed to process image: https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg
1/1 [==============================] - 0s 342ms/step
Failed to process image: https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg
1/1 [==============================] - 0s 215ms/step
Failed to process image: https://images-na.ssl-images-amazon.com/images/I/61OboZT-kcL._SY88.jpg
1/1 [==============================] - 0s 194ms/step
Failed to process image: https://images-na.ssl-images-amazon.com/images/I/710a2Pyh5lL._SY88.jpg
1/1 [==============================] - 0s 193ms/step
Failed to process image: https://images-na.ssl-images-amazon.com/images/I/816NMd0LexL._SY88.jpg
1/1 [==============================] - 0s 198ms/step


In [14]:
from sklearn.preprocessing import StandardScaler

# Convert features_list to a NumPy array for easier manipulation
features_array = np.array(features_list)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to your data and transform it
normalized_features = scaler.fit_transform(features_array)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Assuming df['Review Text'] is already filled with empty strings for NaN values
text_features = tfidf_vectorizer.fit_transform(df['Review Text'])

# If you wish to normalize these text features as well:
# Use the same StandardScaler initialized before or create a new one for text features
scaler_text = StandardScaler(with_mean=False)  # with_mean=False to support sparse matrices
normalized_text_features = scaler_text.fit_transform(text_features)


In [17]:
# Fill NaN values with an empty string
df['Review Text'] = df['Review Text'].fillna('')

# Now, proceed with TF-IDF vectorization
text_features = tfidf_vectorizer.fit_transform(df['Review Text'])

# Continue with normalization if needed


## **Text Feature Extraction**

In [8]:
!pip install nltk scikit-learn


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
def preprocess_text(text):
    # Lower-casing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing punctuation
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    # Removing non-alphabetic tokens and stopwords
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    # Stemming/Lemmatization (optional, choose one)
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed = [stemmer.stem(word) for word in words]
    lemmatized = [lemmatizer.lemmatize(word) for word in words]

    # Choose stemmed or lemmatized for the final text
    return ' '.join(lemmatized)


In [18]:
df['processed_text'] = df['Review Text'].apply(preprocess_text)


In [19]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_text'])


In [20]:
import pickle

# Assuming tfidf_matrix is your TF-IDF matrix and tfidf_vectorizer is the vectorizer
with open('/content/drive/MyDrive/tfidf_matrix.pkl', 'wb') as file:
    pickle.dump(tfidf_matrix, file)

with open('/content/drive/MyDrive/tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)


# **Image Retrieval and Text Retrieval**

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [24]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
import requests
from io import BytesIO

# Initialize the pre-trained model
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)



In [37]:
def download_and_process_img(url):
    response = requests.get(url)
    img = load_img(BytesIO(response.content), target_size=(224, 224))
    img_array = img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)

In [39]:
# Assuming 'Image' column contains image URLs
features_array = []  # This will store all the features

In [40]:
# for index, row in df.iterrows():
#     # Extract the URL from the 'Image' column. Adjust as necessary for your DataFrame structure
#     img_url = row['Image']
#     processed_img = download_and_process_img(img_url)
#     features = model.predict(processed_img)
#     features_array.append(features.flatten())

# features_array = np.array(features_array)  # Convert the list to a numpy array for easier handling

In [32]:
import ast  # Importing ast for safely converting string representations of lists to lists

features_list = []  # Initialize the list to store features

for index, row in df.iterrows():
    try:
        # Assuming 'Image' column contains a list in string format or a direct URL
        if isinstance(row['Image'], str) and row['Image'].startswith('['):
            img_urls = ast.literal_eval(row['Image'])
            img_url = img_urls[0] if img_urls else None
        else:
            img_url = row['Image']  # Direct URL case

        if img_url:  # If a URL is available
            processed_img = download_and_process_img(img_url)
            features = model.predict(processed_img)
            features_list.append(features.flatten())
        else:
            print(f"No valid URL found for row {index}")
            features_list.append(np.zeros((model.output.shape[1],)))  # Placeholder for missing or failed images

    except UnidentifiedImageError:
        print(f"Failed to process image at {img_url}")
        features_list.append(np.zeros((model.output.shape[1],)))  # Placeholder for errors

    except Exception as e:
        print(f"An error occurred for row {index}: {e}")
        features_list.append(np.zeros((model.output.shape[1],)))  # Placeholder for other errors

features_array = np.array(features_list)  # Convert the list to a numpy array


1/1 [==============================] - 0s 307ms/step
Failed to process image at https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg
1/1 [==============================] - 0s 196ms/step
Failed to process image at https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg
1/1 [==============================] - 0s 207ms/step
Failed to process image at https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg
1/1 [==============================] - 0s 338ms/step
Failed to process image at https://images-na.ssl-images-amazon.com/images/I/61OboZT-kcL._SY88.jpg
1/1 [==============================] - 0s 192ms/step
Failed to process image at https://images-na.ssl-images-amazon.com/images/I/710a2Pyh5lL._SY88.jpg
1/1 [==============================] - 0s 341ms/step
Failed to process image at https://images-na.ssl-images-amazon.com/images/I/816NMd0LexL._SY88.jpg
1/1 [==============================] - 0s 193ms/step


In [42]:
import ast
from sklearn.metrics.pairwise import cosine_similarity

# Process the first three images and find their features
input_features = []
for i in range(3):
    img_urls = ast.literal_eval(df.iloc[i]['Image'])
    if img_urls:  # Check if the list is not empty
        img_url = img_urls[0]  # Use the first URL
        processed_img = download_and_process_img(img_url)
        feature_vector = model.predict(processed_img)
        input_features.append(feature_vector)
    else:
        input_features.append(np.zeros((1, model.output.shape[1])))  # Placeholder for missing images

# Convert list of feature vectors to an array
input_features_array = np.vstack(input_features)


1/1 [==============================] - 0s 190ms/step


In [ ]:
# Test with a single, known image URL
test_img_url = 'https://example.com/someimage.jpg'  # Replace with a valid image URL
processed_img = download_and_process_img(test_img_url)
features = model.predict(processed_img)
print(features.shape)


In [52]:
import ast

# Assuming 'df' is your dataframe and it's already loaded
# Convert the string representation of the list to an actual list and select the first URL
first_row_image_urls = ast.literal_eval(df.iloc[0]['Image'])
first_image_url = first_row_image_urls[0] if first_row_image_urls else None

if first_image_url:
    # Process the first image URL
    processed_img = download_and_process_img(first_image_url)
    features = model.predict(processed_img)
    print("Features shape:", features.shape)
else:
    print("No valid URL found in the first row.")


1/1 [==============================] - 0s 354ms/step
Features shape: (1, 2048)


In [54]:
def download_and_process_img(url):
    try:
        response = requests.get(url)
        img = load_img(BytesIO(response.content), target_size=(224, 224))
        img_array = img_to_array(img)
        img_array_expanded = np.expand_dims(img_array, axis=0)
        return preprocess_input(img_array_expanded)
    except Exception as e:
        print(f"Error processing image URL {url}: {e}")
        return None


In [55]:
for index, row in df.iterrows():
    image_urls = ast.literal_eval(row['Image'])
    if image_urls:  # Ensure the list is not empty
        first_image_url = image_urls[0]
        processed_img = download_and_process_img(first_image_url)
        if processed_img is not None:
            features = model.predict(processed_img).flatten()
            features_list.append(features)
        else:
            print(f"Skipping image due to error: {first_image_url}")
            features_list.append(np.zeros((2048,)))  # Use a zero vector as a placeholder
    else:
        print("No image URL found in row, using placeholder.")
        features_list.append(np.zeros((2048,)))  # Placeholder for missing images


1/1 [==============================] - 0s 190ms/step
Error processing image URL https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x78d6ded24e00>
Skipping image due to error: https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg
1/1 [==============================] - 0s 210ms/step
Error processing image URL https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x78d6deb865c0>
Skipping image due to error: https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg
1/1 [==============================] - 0s 185ms/step
Error processing image URL https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x78d6d03ed850>
Skipping image due to error: https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg
1/1 [==============================] - 0s 187m

In [58]:
# Assuming the first image's features are your input features
input_features = features_list[0].reshape(1, -1)  # Reshape to 2D array for cosine_similarity function

# Convert your features_list to a NumPy array if you haven't yet
features_array = np.vstack(features_list)

# Calculate cosine similarity
cosine_sim = cosine_similarity(input_features, features_array)

# Get the indices of the top 3 most similar images, excluding the first one (itself)
# Use argsort to sort the similarity scores and get the indices
similar_indices = np.argsort(cosine_sim[0])[::-1][1:4]  # Skip the first one, take the next top 3

print("Indices of Top 3 Similar Images:", similar_indices)
for i in similar_indices:
    print("Image URL:", df.iloc[i]['Image'])
    print("Cosine Similarity Score:", cosine_sim[0][i])


Indices of Top 3 Similar Images: [ 67 588 847]
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg']
Cosine Similarity Score: 1.0
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/51N6VYcTMEL._SY88.jpg']
Cosine Similarity Score: 0.7112015567302568
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/8128C2ja3vL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81Zwa+ao-3L._SY88.jpg']
Cosine Similarity Score: 0.6889176418952643


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming `df['Review Text']` contains your review texts
reviews = df['Review Text'].values

# Step 1: Generate TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)

# Choose an input review, here we take the first one as an example
input_review_vector = tfidf_vectorizer.transform([reviews[0]])

# Step 2: Calculate Cosine Similarity
cosine_similarities = cosine_similarity(input_review_vector, tfidf_matrix)

# Step 3: Rank and Retrieve Top Reviews
top_review_indices = np.argsort(cosine_similarities[0])[-4:]  # Exclude the first one which is the review itself
top_review_indices = top_review_indices[:-1][::-1]  # Get the top 3 indices, exclude the highest (itself)

print("Indices of Top 3 Similar Reviews:", top_review_indices)
for index in top_review_indices:
    print("Review:", reviews[index])
    print("Similarity Score:", cosine_similarities[0][index])


Indices of Top 3 Similar Reviews: [271 805 936]
Review: Nice solid springs and defeinitely more silent. Easy installation and the black looks cool.

Pictured with some old uninstalled springs next to them.
Similarity Score: 0.27817364429129526
Review: Fits great but the only complaint I have is that the tremolo springs are cheap quality and so is the block that attaches to the bridge. I took it off and put my own on. When you do that though, the wammy bar won't fit.
Similarity Score: 0.2421633796935153
Review: I put these in a new bass I built last week.
They sound awesome with lots of punch and, they stay clean on the low notes. Im impressed with em.
The down side would be:
1. The covers are not connected to the pickups and, the adjustment springs support the covers. So, you need to glue the covers on so the weight of the pickup doesn't pull loose and fall into the cavity over time. I used clear epoxy and, hoping for the best since I sold the bass when it was finished. On a positive n

In [57]:
import pickle

# Example of saving the TF-IDF matrix
with open('tfidf_matrix.pkl', 'wb') as file:
    pickle.dump(tfidf_matrix, file)

# Example of loading the TF-IDF matrix
with open('tfidf_matrix.pkl', 'rb') as file:
    loaded_tfidf_matrix = pickle.load(file)


In [59]:
import os
print(os.getcwd())


/content


In [63]:
import pickle

# Assuming tfidf_matrix and features_array are the objects you want to save
pickle.dump(tfidf_matrix, open(os.path.join(save_directory, 'tfidf_matrix.pkl'), 'wb'))
pickle.dump(features_array, open(os.path.join(save_directory, 'features_array.pkl'), 'wb'))


In [64]:
# Function to print first five entries of an object, adjusting based on the object's type
def print_first_five_entries(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
        # Adjust this logic based on the structure of your object
        # Here's an example for a list or array-like object
        print(obj[:5])

# Paths to your saved files
tfidf_path = os.path.join(save_directory, 'tfidf_matrix.pkl')
features_path = os.path.join(save_directory, 'features_array.pkl')

# Print the first five entries
print("TF-IDF Matrix First Five Entries:")
print_first_five_entries(tfidf_path)

print("\nFeatures Array First Five Entries:")
print_first_five_entries(features_path)


TF-IDF Matrix First Five Entries:
  (0, 2498)	0.1360847475853359
  (0, 5767)	0.05498509416700064
  (0, 6185)	0.14356488024637395
  (0, 5654)	0.11200241784385376
  (0, 3831)	0.06625905977545113
  (0, 3917)	0.10937429043958326
  (0, 3618)	0.14640976312809778
  (0, 5661)	0.09687181972788582
  (0, 6157)	0.14356488024637395
  (0, 896)	0.17322206730395334
  (0, 6383)	0.23512767084622577
  (0, 2283)	0.2578120570362944
  (0, 505)	0.17483983368491277
  (0, 6379)	0.07962090764089085
  (0, 2843)	0.09835139176341018
  (0, 5324)	0.24178531453798852
  (0, 2533)	0.08435824682002493
  (0, 448)	0.09736807761007331
  (0, 5637)	0.22575857203968258
  (0, 2507)	0.10201389979836627
  (0, 2663)	0.07918773277940222
  (0, 5674)	0.09630644556460974
  (0, 5417)	0.16987978344979346
  (0, 3677)	0.06406885771764068
  (0, 3858)	0.06966305963948213
  :	:
  (4, 5331)	0.12023728883759432
  (4, 3462)	0.07241242597319503
  (4, 3661)	0.09938198751340516
  (4, 4489)	0.17549512597860967
  (4, 685)	0.4335494930112185
  (4, 5

# **Combined Retrieval**

In [70]:
# # Assuming image_similarity_scores and text_similarity_scores are lists or arrays of the same length
# composite_scores = [(img_score + txt_score) / 2 for img_score, txt_score in zip(image_similarity_scores, text_similarity_scores)]

# # Rank the pairs based on composite score
# # If you need to keep track of the original indices/pairs, consider using enumerate and sorting by score
# ranked_pairs_indices = sorted(range(len(composite_scores)), key=lambda i: composite_scores[i], reverse=True)

# # Now ranked_pairs_indices contains indices sorted by the highest composite score
# # Use these indices to retrieve or reference the original pairs

# # Example: Print top 3 ranked pairs
# print("Top 3 ranked pairs based on composite similarity score:")
# for idx in ranked_pairs_indices[:3]:
#     print(f"Pair {idx} with composite score: {composite_scores[idx]}")


In [66]:
# Recalculate the image similarity for demonstration
# Choose an input image features vector
input_image_features = features_array[0].reshape(1, -1)  # Example: Taking the first image's features

# Calculate cosine similarity for images
image_similarity_scores = cosine_similarity(input_image_features, features_array)[0]


In [67]:
# Assuming you have an input review's TF-IDF vector
# This would have been generated similar to how you processed all reviews into the tfidf_matrix
# For demonstration, let's assume it's the TF-IDF vector for the first review
input_review_vector = tfidf_matrix[0:1]

# Calculate cosine similarity for text
text_similarity_scores = cosine_similarity(input_review_vector, tfidf_matrix).flatten()


In [68]:
# Calculate composite scores by averaging image and text similarity scores
composite_scores = [(img_score + txt_score) / 2 for img_score, txt_score in zip(image_similarity_scores, text_similarity_scores)]

# Rank the pairs based on composite score
ranked_pairs_indices = sorted(range(len(composite_scores)), key=lambda i: composite_scores[i], reverse=True)

# Example: Print top 3 ranked pairs based on composite similarity score
print("Top 3 ranked pairs based on composite similarity score:")
for idx in ranked_pairs_indices[:3]:
    print(f"Pair {idx} with composite score: {composite_scores[idx]}")


Top 3 ranked pairs based on composite similarity score:
Pair 0 with composite score: 1.0
Pair 67 with composite score: 0.5154488377965074
Pair 271 with composite score: 0.44717076013080903


In [69]:
from IPython.display import display, Image
import ast

# Function to display an image from a URL
def display_image_from_url(url):
    display(Image(url=url))

# Assuming `df` is your DataFrame
for idx in [0, 67, 271]:  # Example indices from your results
    print(f"Pair {idx} with composite score: {composite_scores[idx]}")
    row = df.iloc[idx]
    image_urls = ast.literal_eval(row['Image'])
    if image_urls:
        print("Displaying Image:")
        display_image_from_url(image_urls[0])
    print("Review:", row['Review Text'])
    print("\n---\n")


Pair 0 with composite score: 1.0
Displaying Image:


Review: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.

---

Pair 67 with composite score: 0.5154488377965074
Displaying Image:


Review: Its flimsy and for the awkward angles it sticks in a guitar the Vox device should be metal and sturdy, especially for price.  Mine broke within a week and I am a just a living room player.  I don't know how it broke but it did.  I will upload pictures.

Now aside of the cheaply made product it served its purpose for the time it lasted and provided sound through headphones,  but its better to get a portable amp.

---

Pair 271 with composite score: 0.44717076013080903
Displaying Image:


Review: Nice solid springs and defeinitely more silent. Easy installation and the black looks cool.

Pictured with some old uninstalled springs next to them.

---



In [71]:
# Calculate composite similarity scores as the average of image and text similarity scores
composite_similarity_scores = [(img_score + text_score) / 2 for img_score, text_score in zip(image_similarity_scores, text_similarity_scores)]


In [73]:
# Example scores for demonstration; replace these with your actual similarity scores
# image_similarity_scores = [0.8, 0.5, 0.9]  # Example data
# text_similarity_scores = [0.7, 0.6, 0.4]   # Example data

# Calculate composite similarity scores as the average of image and text similarity scores
composite_similarity_scores = [(img_score + text_score) / 2 for img_score, text_score in zip(image_similarity_scores, text_similarity_scores)]

# Sort the indices of these scores in descending order to rank them from most to least similar
sorted_indices = sorted(range(len(composite_similarity_scores)), key=lambda i: composite_similarity_scores[i], reverse=True)

# Now, use sorted_indices to get the top N similar pairs
top_n = 3  # Adjust based on how many top pairs you'd like to retrieve
top_n_indices = sorted_indices[:top_n]

print(f"Top {top_n} similar pairs (based on composite similarity score):")
for idx in top_n_indices:
    print(f"Pair {idx} with composite score: {composite_similarity_scores[idx]}")
    # Optionally, display or process the corresponding image and text pair as needed


Top 3 similar pairs (based on composite similarity score):
Pair 0 with composite score: 1.0
Pair 67 with composite score: 0.5154488377965074
Pair 271 with composite score: 0.44717076013080903


# **Results & Analysis**

In [74]:
from IPython.display import display, Image
import ast

# Function to display an image from a URL
def display_image_from_url(url):
    display(Image(url=url))

# Display top-ranked pairs
for idx in [0, 67, 271]:
    # Assuming you have a dataframe `df` with 'Image' and 'Review Text' columns
    image_urls = ast.literal_eval(df.iloc[idx]['Image'])
    review_text = df.iloc[idx]['Review Text']
    composite_score = composite_similarity_scores[idx]

    print(f"Pair {idx} with composite score: {composite_score}")
    if image_urls:  # Check if there are any image URLs
        display_image_from_url(image_urls[0])  # Display the first image
    print("Review:", review_text)
    print("\n---\n")


Pair 0 with composite score: 1.0


Review: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.

---

Pair 67 with composite score: 0.5154488377965074


Review: Its flimsy and for the awkward angles it sticks in a guitar the Vox device should be metal and sturdy, especially for price.  Mine broke within a week and I am a just a living room player.  I don't know how it broke but it did.  I will upload pictures.

Now aside of the cheaply made product it served its purpose for the time it lasted and provided sound through headphones,  but its better to get a portable amp.

---

Pair 271 with composite score: 0.44717076013080903


Review: Nice solid springs and defeinitely more silent. Easy installation and the black looks cool.

Pictured with some old uninstalled springs next to them.

---



# **Sample Test Cases**

In [76]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# Initialize the pre-trained model
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

def download_and_process_img(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).resize((224, 224))
    img_array = img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)

def extract_features(processed_img):
    return model.predict(processed_img)


In [77]:
# Assuming the following functions are defined and models are loaded:
# - download_and_process_img(img_url) -> processes the image for the model
# - extract_features(image) -> returns the feature vector for an image
# - tfidf_vectorizer -> TF-IDF vectorizer fitted on your dataset
# - image_similarity_scores, text_similarity_scores -> arrays/lists containing pre-calculated scores

# Input
input_img_url = "https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg"
input_review_text = "I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break."

# Process the input image and text
input_img_processed = download_and_process_img(input_img_url)
input_img_features = extract_features(input_img_processed).flatten()
input_text_vector = tfidf_vectorizer.transform([input_review_text])

# Calculate similarities
image_similarity_scores = cosine_similarity(input_img_features.reshape(1, -1), features_array)
text_similarity_scores = cosine_similarity(input_text_vector, tfidf_matrix)

# Combine and rank (simplified version, real implementation will depend on your data structures)
composite_scores = [(img_score + txt_score) / 2 for img_score, txt_score in zip(image_similarity_scores.flatten(), text_similarity_scores.flatten())]
sorted_indices = sorted(range(len(composite_scores)), key=lambda i: composite_scores[i], reverse=True)

# Output for top 3
for idx in sorted_indices[:3]:
    print(f"{idx+1}) Image URL: {df.iloc[idx]['Image']}")
    print(f"Review: {df.iloc[idx]['Review Text']}")
    print(f"Cosine similarity of images - {image_similarity_scores[0][idx]:.4f}")
    print(f"Cosine similarity of text - {text_similarity_scores[0][idx]:.4f}")
    print("---")

# Composite Scores (example, adjust according to how you calculate/store these)
print(f"Composite similarity scores of images: {np.mean(image_similarity_scores):.4f}")
print(f"Composite similarity scores of text: {np.mean(text_similarity_scores):.4f}")
final_composite_score = np.mean(composite_scores)
print(f"Final composite similarity score: {final_composite_score:.4f}")


1/1 [==============================] - 2s 2s/step
759) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg']
Review: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
Cosine similarity of images - 0.3537
Cosine similarity of text - 1.0000
---
623) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/61DvLcapd8L._SY88.jpg']
Review: I went from fender chrome non-locking to fender gold locking. It made my guitar look beautiful and play beautiful. I think locking tuners are the way to go. If you are new to locking tuners look on YouTube for instructions.
Cosine similarity of images - 0.4023
Cosine similarity of text - 0.3032
---
826) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71Cuc+MWuYL._SY88.jpg']
Review: After a string change the tips next to the end started chipping away.
Cosine similarity of images - 